In [1]:
import os
import numpy as np

import tables

from tables import *

In this section, we will learn how to browse the tree and retrieve data and also meta-information about the actual data.
In examples/tutorial1-2.py you will find the working version of all the code in this section. As before, you are encouraged to use a python shell and inspect the object tree during the course of the tutorial.

# Traversing the object tree
Let’s start by opening the file we created in last tutorial section:

In [2]:
h5file = open_file("tmp/tutorial1.h5", "a")

This time, we have opened the file in “a”ppend mode. We use this mode to add more information to the file.
PyTables, following the Python tradition, offers powerful introspection capabilities, i.e. you can easily ask information about any component of the object tree as well as search the tree.
To start with, you can get a preliminary overview of the object tree by simply printing the existing File instance:

In [3]:
print(h5file)

tmp/tutorial1.h5 (File) 'Test file'
Last modif.: 'Mon Sep 23 12:52:43 2019'
Object Tree: 
/ (RootGroup) 'Test file'
/columns (Group) 'Pressure and Name'
/columns/name (Array(3,)) 'Name column selection'
/columns/pressure (Array(3,)) 'Pressure column selection'
/detector (Group) 'Detector information'
/detector/readout (Table(10,)) 'Readout example'
/gabes_group (Group) 'gabes random group'
/gabes_group/gabes_table (Table(0,)) 'Gabe Table Example'



In [4]:
# It looks like all of our objects are there. Now let’s make use of the File 
# iterator to see how to list all the nodes in the object tree:
for node in h5file:
    print(node)

/ (RootGroup) 'Test file'
/columns (Group) 'Pressure and Name'
/detector (Group) 'Detector information'
/gabes_group (Group) 'gabes random group'
/columns/name (Array(3,)) 'Name column selection'
/columns/pressure (Array(3,)) 'Pressure column selection'
/detector/readout (Table(10,)) 'Readout example'
/gabes_group/gabes_table (Table(0,)) 'Gabe Table Example'


In [5]:
# We can use the File.walk_groups() method of the File class to list only the groups on tree:
for group in h5file.walk_groups():
    print(group)

/ (RootGroup) 'Test file'
/columns (Group) 'Pressure and Name'
/detector (Group) 'Detector information'
/gabes_group (Group) 'gabes random group'


In [6]:
# Note that File.walk_groups() actually returns an iterator, not a list of objects. 
# Using this iterator with the list_nodes() method is a powerful combination. 
# Let’s see an example listing of all the arrays in the tree:
for group in h5file.walk_groups("/"):
    for array in h5file.list_nodes(group, classname='Array'):
        print(array)

/columns/name (Array(3,)) 'Name column selection'
/columns/pressure (Array(3,)) 'Pressure column selection'


`File.list_nodes()` returns a list containing all the nodes hanging off a specific Group. If the classname keyword is specified, the method will filter out all instances which are not descendants of the class. We have asked for only Array instances. There exist also an iterator counterpart called File.iter_nodes() that might be handy is some situations, like for example when dealing with groups with a large number of nodes behind it.

In [7]:
# We can combine both calls by using the File.walk_nodes() special method of the File object. For example:
for array in h5file.walk_nodes("/", "Array"):
    print(array)

/columns/name (Array(3,)) 'Name column selection'
/columns/pressure (Array(3,)) 'Pressure column selection'


We have used a call to the `Group._f_walknodes()` method, using the natural naming path specification.
Of course you can do more sophisticated node selections using these powerful methods. But first, let’s take a look at some important PyTables object instance variables.

# Setting and getting user attributes
PyTables provides an easy and concise way to complement the meaning of your node objects on the tree by using the AttributeSet class (see The AttributeSet class). You can access this object through the standard attribute attrs in Leaf nodes and _v_attrs in Group nodes.

For example, let’s imagine that we want to save the date indicating when the data in /detector/readout table has been acquired, as well as the temperature during the gathering process:

In [8]:
table = h5file.root.detector.readout
table.attrs.gath_date = "Wed, 06/12/2003 18:33"
table.attrs.temperature = 18.4
table.attrs.temp_scale = "Celsius"
table.attrs.is_gaby_the_best = 'Hell yeah he is!'

In [9]:
table.attrs

/detector/readout._v_attrs (AttributeSet), 25 attributes:
   [CLASS := 'TABLE',
    FIELD_0_FILL := 0,
    FIELD_0_NAME := 'ADCcount',
    FIELD_1_FILL := 0,
    FIELD_1_NAME := 'TDCcount',
    FIELD_2_FILL := 0.0,
    FIELD_2_NAME := 'energy',
    FIELD_3_FILL := 0,
    FIELD_3_NAME := 'grid_i',
    FIELD_4_FILL := 0,
    FIELD_4_NAME := 'grid_j',
    FIELD_5_FILL := 0,
    FIELD_5_NAME := 'idnumber',
    FIELD_6_FILL := b'',
    FIELD_6_NAME := 'name',
    FIELD_7_FILL := 0.0,
    FIELD_7_NAME := 'pressure',
    NROWS := 10,
    TITLE := 'Readout example',
    VERSION := '2.7',
    gath_date := 'Wed, 06/12/2003 18:33',
    is_gaby_the_best := 'Hell yeah he is!',
    tempScale := 'Celsius',
    temp_scale := 'Celsius',
    temperature := 18.4]

In [10]:
# Now, let’s set a somewhat more complex attribute in the /detector group:
detector = h5file.root.detector
detector._v_attrs.stuff = [5, (2.3, 4.5), "Integer and tuple"]

In [11]:
detector._v_attrs

/detector._v_attrs (AttributeSet), 4 attributes:
   [CLASS := 'GROUP',
    TITLE := 'Detector information',
    VERSION := '1.0',
    stuff := [5, (2.3, 4.5), 'Integer and tuple']]

Note how the AttributeSet instance is accessed with the _v_attrs attribute because detector is a Group node. In general, you can save any standard Python data structure as an attribute node. See The AttributeSet class for a more detailed explanation of how they are serialized for export to disk.

Retrieving the attributes is equally simple:

In [12]:
print('table.attrs.gath_date := %s' % table.attrs.gath_date)
print('table.attrs.temperature := %s' % table.attrs.temperature)
print('table.attrs.temp_scale := %s' % table.attrs.temp_scale)
print('detector._v_attrs.stuff := %s' % detector._v_attrs.stuff)

table.attrs.gath_date := Wed, 06/12/2003 18:33
table.attrs.temperature := 18.4
table.attrs.temp_scale := Celsius
detector._v_attrs.stuff := [5, (2.3, 4.5), 'Integer and tuple']


In [13]:
# You can probably guess how to delete attributes:
del table.attrs.gath_date

If you want to examine the current user attribute set of /detector/table, you can print its representation (try hitting the TAB key twice if you are on a Unix Python console with the rlcompleter module active):

In [14]:
table.attrs

/detector/readout._v_attrs (AttributeSet), 24 attributes:
   [CLASS := 'TABLE',
    FIELD_0_FILL := 0,
    FIELD_0_NAME := 'ADCcount',
    FIELD_1_FILL := 0,
    FIELD_1_NAME := 'TDCcount',
    FIELD_2_FILL := 0.0,
    FIELD_2_NAME := 'energy',
    FIELD_3_FILL := 0,
    FIELD_3_NAME := 'grid_i',
    FIELD_4_FILL := 0,
    FIELD_4_NAME := 'grid_j',
    FIELD_5_FILL := 0,
    FIELD_5_NAME := 'idnumber',
    FIELD_6_FILL := b'',
    FIELD_6_NAME := 'name',
    FIELD_7_FILL := 0.0,
    FIELD_7_NAME := 'pressure',
    NROWS := 10,
    TITLE := 'Readout example',
    VERSION := '2.7',
    is_gaby_the_best := 'Hell yeah he is!',
    tempScale := 'Celsius',
    temp_scale := 'Celsius',
    temperature := 18.4]

We’ve got all the attributes (including the system attributes). You can get a list of all attributes or only the user or system attributes with the `_f_list()` method:

In [15]:
print(table.attrs._f_list("all"))

['CLASS', 'FIELD_0_FILL', 'FIELD_0_NAME', 'FIELD_1_FILL', 'FIELD_1_NAME', 'FIELD_2_FILL', 'FIELD_2_NAME', 'FIELD_3_FILL', 'FIELD_3_NAME', 'FIELD_4_FILL', 'FIELD_4_NAME', 'FIELD_5_FILL', 'FIELD_5_NAME', 'FIELD_6_FILL', 'FIELD_6_NAME', 'FIELD_7_FILL', 'FIELD_7_NAME', 'NROWS', 'TITLE', 'VERSION', 'is_gaby_the_best', 'tempScale', 'temp_scale', 'temperature']


In [16]:
print(table.attrs._f_list("user"))

['is_gaby_the_best', 'tempScale', 'temp_scale', 'temperature']


In [17]:
print(table.attrs._f_list("sys"))

['CLASS', 'FIELD_0_FILL', 'FIELD_0_NAME', 'FIELD_1_FILL', 'FIELD_1_NAME', 'FIELD_2_FILL', 'FIELD_2_NAME', 'FIELD_3_FILL', 'FIELD_3_NAME', 'FIELD_4_FILL', 'FIELD_4_NAME', 'FIELD_5_FILL', 'FIELD_5_NAME', 'FIELD_6_FILL', 'FIELD_6_NAME', 'FIELD_7_FILL', 'FIELD_7_NAME', 'NROWS', 'TITLE', 'VERSION']


In [18]:
# You can also rename attributes:
table.attrs._f_rename("temp_scale","tempScale")
print(table.attrs._f_list())

['is_gaby_the_best', 'tempScale', 'temperature']


In [27]:
# And, from PyTables 2.0 on, you are allowed also to set, delete or rename system attributes:
version = None
try:
    table.attrs._f_rename("VERSION", "version")
    version = table.attrs.version
except:
    table.attrs._f_rename("version", "VERSION")
    version = table.attrs.VERSION

version

'2.7'

Ok. that’s better. If you would terminate your session now, you would be able to use the h5ls command to read the /detector/readout attributes from the file written to disk.

In [28]:
h5file.close()

In [29]:
! h5ls -vr tmp/tutorial1.h5/detector/readout

Opened "tmp/tutorial1.h5" with sec2 driver.
detector/readout         Dataset {10/Inf}
    Attribute: CLASS scalar
        Type:      5-byte null-terminated UTF-8 string
        Data:  "TABLE"
    Attribute: FIELD_0_FILL scalar
        Type:      native unsigned short
        Data:  0
    Attribute: FIELD_0_NAME scalar
        Type:      8-byte null-terminated UTF-8 string
        Data:  "ADCcount"
    Attribute: FIELD_1_FILL scalar
        Type:      native unsigned char
        Data:  0
    Attribute: FIELD_1_NAME scalar
        Type:      8-byte null-terminated UTF-8 string
        Data:  "TDCcount"
    Attribute: FIELD_2_FILL scalar
        Type:      native double
        Data:  0
    Attribute: FIELD_2_NAME scalar
        Type:      6-byte null-terminated UTF-8 string
        Data:  "energy"
    Attribute: FIELD_3_FILL scalar
        Type:      native int
        Data:  0
    Attribute: FIELD_3_NAME scalar
        Type:      6-byte null-terminated UTF-8 string
        Data:  "grid

Attributes are a useful mechanism to add persistent (meta) information to your data.

# Getting object metadata
Each object in PyTables has metadata information about the data in the file. Normally this meta-information is accessible through the node instance variables. Let’s take a look at some examples:

In [40]:
h5file = open_file('tmp/tutorial1.h5', 'a')
table = h5file.root.detector.readout
table

/detector/readout (Table(10,)) 'Readout example'
  description := {
  "ADCcount": UInt16Col(shape=(), dflt=0, pos=0),
  "TDCcount": UInt8Col(shape=(), dflt=0, pos=1),
  "energy": Float64Col(shape=(), dflt=0.0, pos=2),
  "grid_i": Int32Col(shape=(), dflt=0, pos=3),
  "grid_j": Int32Col(shape=(), dflt=0, pos=4),
  "idnumber": Int64Col(shape=(), dflt=0, pos=5),
  "name": StringCol(itemsize=16, shape=(), dflt=b'', pos=6),
  "pressure": Float32Col(shape=(), dflt=0.0, pos=7)}
  byteorder := 'little'
  chunkshape := (1394,)

In [44]:
# Here, the name, title, nrows, colnames and coldtypes attributes (see Table for a complete attribute list) 
# of the Table object gives us quite a bit of information about the table data.
print("Object:", table)
print("Table name:", table.name)
print("Table title:", table.title)
print("Number of rows in table:", table.nrows)
print("Table variable names with their type and shape:")

for name in table.colnames:
    print(name, ':= %s, %s' % (table.coldtypes[name], table.coldtypes[name].shape))

Object: /detector/readout (Table(10,)) 'Readout example'
Table name: readout
Table title: Readout example
Number of rows in table: 10
Table variable names with their type and shape:
ADCcount := uint16, ()
TDCcount := uint8, ()
energy := float64, ()
grid_i := int32, ()
grid_j := int32, ()
idnumber := int64, ()
name := |S16, ()
pressure := float32, ()


In [45]:
# You can interactively retrieve general information about the public objects in PyTables by asking for help:
help(table)

Help on Table in module tables.table object:

class Table(tables.tableextension.Table, tables.leaf.Leaf)
 |  This class represents heterogeneous datasets in an HDF5 file.
 |  
 |  Tables are leaves (see the Leaf class in :ref:`LeafClassDescr`) whose data
 |  consists of a unidimensional sequence of *rows*, where each row contains
 |  one or more *fields*.  Fields have an associated unique *name* and
 |  *position*, with the first field having position 0.  All rows have the same
 |  fields, which are arranged in *columns*.
 |  
 |  Fields can have any type supported by the Col class (see
 |  :ref:`ColClassDescr`) and its descendants, which support multidimensional
 |  data.  Moreover, a field can be *nested* (to an arbitrary depth), meaning
 |  that it includes further fields inside.  A field named x inside a nested
 |  field a in a table can be accessed as the field a/x (its *path name*) from
 |  the table.
 |  
 |  The structure of a table is declared by its description, which is made

In [47]:
# Search the file tree for the "columns" node and extract the pressure column object 
pressureObject = h5file.get_node(where="/columns", name="pressure")
print("Info on the object:", repr(pressureObject))
print("  shape: ==>", pressureObject.shape)
print("  title: ==>", pressureObject.title)
print("  atom: ==>", pressureObject.atom)

Info on the object: /columns/pressure (Array(3,)) 'Pressure column selection'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'python'
  byteorder := 'little'
  chunkshape := None
  shape: ==> (3,)
  title: ==> Pressure column selection
  atom: ==> Float64Atom(shape=(), dflt=0.0)


Observe that we have used the `File.get_node()` method of the File class to access a node in the tree, instead of the natural naming method. Both are useful, and depending on the context you will prefer one or the other. `File.get_node()` has the advantage that it can get a node from the pathname string (as in this example) and can also act as a filter to show only nodes in a particular location that are instances of class classname. In general, however, I consider natural naming to be more elegant and easier to use, especially if you are using the name completion capability present in interactive console. Try this powerful combination of natural naming and completion capabilities present in most Python consoles, and see how pleasant it is to browse the object tree (well, as pleasant as such an activity can be).

If you look at the type attribute of the `pressureObject` object, you can verify that it is a `float64` array. By looking at its shape attribute, you can deduce that the array on disk is unidimensional and has 3 elements. See Array or the internal doc strings for the complete Array attribute list.

# Reading data from Array objects
Once you have found the desired Array, use the `read()` method of the Array object to retrieve its data:

In [48]:
pressureArray = pressureObject.read()
pressureArray

[25.0, 36.0, 49.0]

In [49]:
print("pressureArray is an object of type:", type(pressureArray))

pressureArray is an object of type: <class 'list'>


In [55]:
nameArray = h5file.root.columns.name.read()
print("nameArray is an object of type:", type(nameArray))

nameArray is an object of type: <class 'list'>


In [56]:
print("Data on arrays nameArray and pressureArray:")
for i in range(pressureObject.shape[0]):
    print(nameArray[i], "-->", pressureArray[i])

Data on arrays nameArray and pressureArray:
b'Particle:      5' --> 25.0
b'Particle:      6' --> 36.0
b'Particle:      7' --> 49.0


You can see that the `Array.read()` method returns an authentic NumPy object for the `pressureObject` instance by looking at the output of the `type()` call. A `read()` of the nameArray object instance returns a native Python list (of strings). The type of the object saved is stored as an HDF5 attribute (named FLAVOR) for objects on disk. This attribute is then read as Array meta-information (accessible through in the Array.attrs.FLAVOR variable), enabling the read array to be converted into the original object. This provides a means to save a large variety of objects as arrays with the guarantee that you will be able to later recover them in their original form. See `File.create_array()` for a complete list of supported objects for the Array object class.

# Commiting data to tables and arrays
We have seen how to create tables and arrays and how to browse both data and metadata in the object tree. Let’s examine more closely now one of the most powerful capabilities of PyTables, namely, how to modify already created tables and arrays.

# Appending data to an existing table
Now, let’s have a look at how we can add records to an existing table on disk. Let’s use our well-known readout Table object and append some new values to it:

In [68]:
table = h5file.root.detector.readout
particle = table.row

for i in range(10, 15):
    particle['name']  = 'Particle: %6d' % (i)
    particle['TDCcount'] = i % 256
    particle['ADCcount'] = (i * 256) % (1 << 16)
    particle['grid_i'] = i
    particle['grid_j'] = 10 - i
    particle['pressure'] = float(i*i)
    particle['energy'] = float(particle['pressure'] ** 4)
    particle['idnumber'] = i * (2 ** 34)
    particle.append()

table.flush()

It’s the same method we used to fill a new table. PyTables knows that this table is on disk, and when you add new records, they are appended to the end of the table.

If you look carefully at the code you will see that we have used the table.row attribute to create a table row and fill it with the new values. Each time that its `append()` method is called, the actual row is committed to the output buffer and the row pointer is incremented to point to the next table record. When the buffer is full, the data is saved on disk, and the buffer is reused again for the next cycle.

_Caveat emptor:_ Do not forget to always call the flush() method after a write operation, or else your tables will not be updated!

In [71]:
# Let’s have a look at some rows in the modified table and verify that our new data has been appended:
for r in table.iterrows():
    print("%-16s | %11.1f | %11.4g | %6d | %6d | %8d \|" % \
          (r['name'], r['pressure'], r['energy'], r['grid_i'], r['grid_j'],
           r['TDCcount']))

b'Particle:      0' |         0.0 |           0 |      0 |     10 |        0 \|
b'Particle:      1' |         1.0 |           1 |      1 |      9 |        1 \|
b'Particle:      2' |         4.0 |         256 |      2 |      8 |        2 \|
b'Particle:      3' |         9.0 |        6561 |      3 |      7 |        3 \|
b'Particle:      4' |        16.0 |   6.554e+04 |      4 |      6 |        4 \|
b'Particle:      5' |        25.0 |   3.906e+05 |      5 |      5 |        5 \|
b'Particle:      6' |        36.0 |    1.68e+06 |      6 |      4 |        6 \|
b'Particle:      7' |        49.0 |   5.765e+06 |      7 |      3 |        7 \|
b'Particle:      8' |        64.0 |   1.678e+07 |      8 |      2 |        8 \|
b'Particle:      9' |        81.0 |   4.305e+07 |      9 |      1 |        9 \|
b'Particle:     10' |       100.0 |       1e+08 |     10 |      0 |       10 \|
b'Particle:     11' |       121.0 |   2.144e+08 |     11 |     -1 |       11 \|
b'Particle:     12' |       144.0 |     

# Modifying data in tables
Ok, until now, we’ve been only reading and writing (appending) values to our tables. But there are times that you need to modify your data once you have saved it on disk (this is specially true when you need to modify the real world data to adapt your goals ;). Let’s see how we can modify the values that were saved in our existing tables. We will start modifying single cells in the first row of the Particle table:

In [72]:
print("Before modif-->", table[0])

table.cols.TDCcount[0] = 1
print("After modifying first row of ADCcount-->", table[0])

table.cols.energy[0] = 2
print("After modifying first row of energy-->", table[0])

Before modif--> (0, 0, 0., 0, 10, 0, b'Particle:      0', 0.)
After modifying first row of ADCcount--> (0, 1, 0., 0, 10, 0, b'Particle:      0', 0.)
After modifying first row of energy--> (0, 1, 2., 0, 10, 0, b'Particle:      0', 0.)


In [75]:
# We can modify complete ranges of columns as well:
table.cols.TDCcount[2:5] = [2,3,4]
print("After modifying slice [2:5] of TDCcount-->", table[0:5])

table.cols.energy[1:9:3] = [2,3,4]
print("After modifying slice [1:9:3] of energy-->", table[0:9])

After modifying slice [2:5] of TDCcount--> [(   0, 1, 2.000e+00, 0, 10,           0, b'Particle:      0',  0.)
 ( 256, 1, 2.000e+00, 1,  9, 17179869184, b'Particle:      1',  1.)
 ( 512, 2, 2.560e+02, 2,  8, 34359738368, b'Particle:      2',  4.)
 ( 768, 3, 6.561e+03, 3,  7, 51539607552, b'Particle:      3',  9.)
 (1024, 4, 3.000e+00, 4,  6, 68719476736, b'Particle:      4', 16.)]
After modifying slice [1:9:3] of energy--> [(   0, 1, 2.0000000e+00, 0, 10,            0, b'Particle:      0',  0.)
 ( 256, 1, 2.0000000e+00, 1,  9,  17179869184, b'Particle:      1',  1.)
 ( 512, 2, 2.5600000e+02, 2,  8,  34359738368, b'Particle:      2',  4.)
 ( 768, 3, 6.5610000e+03, 3,  7,  51539607552, b'Particle:      3',  9.)
 (1024, 4, 3.0000000e+00, 4,  6,  68719476736, b'Particle:      4', 16.)
 (1280, 5, 3.9062500e+05, 5,  5,  85899345920, b'Particle:      5', 25.)
 (1536, 6, 1.6796160e+06, 6,  4, 103079215104, b'Particle:      6', 36.)
 (1792, 7, 4.0000000e+00, 7,  3, 120259084288, b'Particle:    

**Remember** that column TDCcount is the second one, and that energy is the third. Look for more info on modifying columns in `Column.__setitem__()`.

PyTables also lets you modify complete sets of rows at the same time. As a demonstration of these capability, see the next example:

In [78]:
table.modify_rows(
    start=1, step=3,
    rows=[(1, 2, 3.0, 4, 5, 6, 'Particle:   None', 8.0),
          (2, 4, 6.0, 8, 10, 12, 'Particle: None*2', 16.0)])

2

In [79]:
print("After modifying the complete third row-->", table[0:5])

After modifying the complete third row--> [(  0, 1, 2.000e+00, 0, 10,           0, b'Particle:      0',  0.)
 (  1, 2, 3.000e+00, 4,  5,           6, b'Particle:   None',  8.)
 (512, 2, 2.560e+02, 2,  8, 34359738368, b'Particle:      2',  4.)
 (768, 3, 6.561e+03, 3,  7, 51539607552, b'Particle:      3',  9.)
 (  2, 4, 6.000e+00, 8, 10,          12, b'Particle: None*2', 16.)]


As you can see, the `modify_rows()` call has modified the rows second and fifth, and it returned the number of modified rows.

Apart of `Table.modify_rows()`, there exists another method, called `Table.modify_column()` to modify specific columns as well.

Finally, it exists another way of modifying tables that is generally more handy than the described above. This new way uses the method `Row.update()` of the Row instance that is attached to every table, so it is meant to be used in table iterators. Look at the next example:

In [82]:
for row in table.where('TDCcount <= 2'):
    row['energy'] = row['TDCcount']*2
    row.update()

print("After modifying energy column (where TDCcount <=2) -->\n", table[0:4])

After modifying energy column (where TDCcount <=2) -->
 [(  0, 1, 2.000e+00, 0, 10,           0, b'Particle:      0', 0.)
 (  1, 2, 4.000e+00, 4,  5,           6, b'Particle:   None', 8.)
 (512, 2, 4.000e+00, 2,  8, 34359738368, b'Particle:      2', 4.)
 (768, 3, 6.561e+03, 3,  7, 51539607552, b'Particle:      3', 9.)]


**NOTE:** The authors find this way of updating tables (i.e. using `Row.update()`) to be both convenient and efficient. Please make sure to use it extensively.

_Caveat emptor:_ Currently, `Row.update()` will not work (the table will not be updated) if the loop is broken with break statement. A possible workaround consists in manually flushing the row internal buffer by calling `row._flushModRows()` just before the break statement.

# Modifying data in arrays
We are going now to see how to modify data in array objects. The basic way to do this is through the use of `Array.__setitem__()` special method. Let’s see at how modify data on the `pressureObject` array:

In [84]:
pressureObject = h5file.root.columns.pressure
print("Before modif -->", pressureObject[:])

pressureObject[0] = 2
print("First modif -->", pressureObject[:])

pressureObject[1:3] = [2.1, 3.5]
print("Second modif -->", pressureObject[:])

pressureObject[::2] = [1,2]
print("Third modif -->", pressureObject[:])

Before modif --> [1.0, 2.1, 2.0]
First modif --> [2.0, 2.1, 2.0]
Second modif --> [2.0, 2.1, 3.5]
Third modif --> [1.0, 2.1, 2.0]


So, in general, you can use any combination of (multidimensional) extended slicing.
With the sole exception that you cannot use negative values for step to refer to indexes that you want to modify. See `Array.__getitem__()` for more examples on how to use extended slicing in PyTables objects.

In [97]:
# Similarly, with an array of strings:
nameObject = h5file.root.columns.name
print("Before modif -->", nameObject[:])

nameObject[0] = b'Particle:   None'
print("First modif -->", nameObject[:])

nameObject[1:3] = ['Particle:      0', 'Particle:      1']
print("Second modif -->", nameObject[:])

nameObject[::2] = ['Particle:     -3', 'Particle:     -5']
print("Third modif -->", nameObject[:])

Before modif --> [b'Particle:     -3', b'Particle:      0', b'Particle:     -5']
First modif --> [b'Particle:   None', b'Particle:      0', b'Particle:     -5']
Second modif --> [b'Particle:   None', b'Particle:      0', b'Particle:      1']
Third modif --> [b'Particle:     -3', b'Particle:      0', b'Particle:     -5']


# And finally… how to delete rows from a table
We’ll finish this tutorial by deleting some rows from the table we have. Suppose that we want to delete the 5th to 9th rows (inclusive):

In [98]:
table.remove_rows(5,10)

5

`Table.remove_rows()` deletes the rows in the range (start, stop). It returns the number of rows effectively removed.

We have reached the end of this first tutorial. Don’t forget to close the file when you finish:

In [99]:
 h5file.close()

In [ ]:
! 